In [4]:
import pandas as pd
import scipy.signal as signal
import numpy as np
import os

In [5]:
# -------------------------
# SensorID to Body Part Map
# -------------------------
SENSOR_MAP = {
    1: "LT",
    2: "RT",
    3: "LS",
    4: "RS",
    5: "LA",
    6: "RA"
}

In [6]:
# # -------------------------
# # Load IMU data
# # -------------------------
# imu_df = pd.read_csv("dataset with force/Test_13-3-2025/Specimen_1/IMU/test1.csv")  # path to your combined IMU file
# imu_df = imu_df.sort_values("Time(ms)").reset_index(drop=True)

base_path = "dataset with force/Test_13-3-2025/Specimen_1"
imu_path = os.path.join(base_path, "IMU")
left_force_path = os.path.join(base_path, "Left_Force")
right_force_path = os.path.join(base_path, "Right_Force")

In [7]:
# Resample function
def resample_force(force_df, target_len):
    force = force_df.sort_values("Time(ms)")["ForceValue"].values
    return signal.resample(force, target_len)

# Get list of all test files (assuming all folders have same filenames)
test_files = sorted([f for f in os.listdir(imu_path) if f.endswith(".csv")])


In [8]:
print(test_files)

['test1.csv', 'test10.csv', 'test2.csv', 'test3.csv', 'test4.csv', 'test5.csv', 'test6.csv', 'test7.csv', 'test8.csv', 'test9.csv']


In [9]:
for test_file in test_files:
    test_name = test_file.split(".csv")[0]
    print(f"Processing {test_name}...")

    # Load IMU data
    imu_df = pd.read_csv(os.path.join(imu_path, test_file))
    imu_df = imu_df.sort_values("Time(ms)").reset_index(drop=True)

    # Pivot IMU to wide format
    imu_wide = {}
    for sensor_id, label in SENSOR_MAP.items():
        df_part = imu_df[imu_df["SensorID"] == sensor_id].copy()
        df_part = df_part.rename(columns={
            "AccelX": f"{label}_AccelX",
            "AccelY": f"{label}_AccelY",
            "AccelZ": f"{label}_AccelZ",
            "GyroX": f"{label}_GyroX",
            "GyroY": f"{label}_GyroY",
            "GyroZ": f"{label}_GyroZ",
            "Time(ms)": "Time"
        })
        df_part = df_part.drop(columns=["SensorID"])
        imu_wide[label] = df_part

    # Merge all sensors by time
    combined_imu = imu_wide["RS"]
    for part in ["RA", "RT", "LT", "LA", "LS"]:
        combined_imu = pd.merge_asof(combined_imu, imu_wide[part], on="Time", direction="nearest")

    # Load force data
    left_force = pd.read_csv(os.path.join(left_force_path, test_file))
    right_force = pd.read_csv(os.path.join(right_force_path, test_file))

    # Add force data to IMU dataframe
    combined_imu["Left_Force"] = resample_force(left_force, len(combined_imu))
    combined_imu["Right_Force"] = resample_force(right_force, len(combined_imu))

    # Save combined file
    out_path = os.path.join(base_path, f"combined_dataset_{test_name}.csv")
    combined_imu.to_csv(out_path, index=False)
    print(f"Saved {out_path}")

Processing test1...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test1.csv
Processing test10...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test10.csv
Processing test2...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test2.csv
Processing test3...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test3.csv
Processing test4...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test4.csv
Processing test5...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test5.csv
Processing test6...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test6.csv
Processing test7...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test7.csv
Processing test8...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test8.csv
Processing test9...
Saved dataset with force/Test_13-3-2025/Specimen_1/combined_dataset_test9.csv


In [ ]:
# # Pivot to wide format for each body part
# imu_wide = {}
# for sensor_id, label in SENSOR_MAP.items():
#     df_part = imu_df[imu_df["SensorID"] == sensor_id].copy()
#     df_part = df_part.rename(columns={
#         "AccelX": f"{label}_AccelX",
#         "AccelY": f"{label}_AccelY",
#         "AccelZ": f"{label}_AccelZ",
#         "GyroX": f"{label}_GyroX",
#         "GyroY": f"{label}_GyroY",
#         "GyroZ": f"{label}_GyroZ",
#         "Time(ms)": "Time"
#     })
#     df_part = df_part.drop(columns=["SensorID"])
#     imu_wide[label] = df_part

In [ ]:
# # Merge all body part IMU data on time
# combined_imu = imu_wide["RS"]
# for part in ["RA", "RT", "LT", "LA", "LS"]:
#     combined_imu = pd.merge_asof(combined_imu, imu_wide[part], on="Time", direction="nearest")


In [ ]:
# # -------------------------
# # Load Force Sensor Data
# # -------------------------
# left_force = pd.read_csv("dataset with force/Test_13-3-2025/Specimen_1/Left_Force/test1.csv")   # path to left force sensor
# right_force = pd.read_csv("dataset with force/Test_13-3-2025/Specimen_1/Right_Force/test1.csv") # path to right force sensor


In [ ]:
# # Resample to match combined_imu length
# def resample_force(force_df, target_len):
#     force = force_df.sort_values("Time(ms)")["ForceValue"].values
#     return signal.resample(force, target_len)

# combined_imu["Left_Force"] = resample_force(left_force, len(combined_imu))
# combined_imu["Right_Force"] = resample_force(right_force, len(combined_imu))

In [ ]:
# # -------------------------
# # Final Output
# # -------------------------
# combined_imu.to_csv("combined_dataset.csv", index=False)
# print("Saved combined_dataset.csv")


Saved combined_dataset.csv
